#### 1.Intro – ein Einfacher Agent & Die Einheiten von MicroRTS

In [ ]:
public class RandomAI extends AI {    
    public RandomAI(UnitTypeTable utt) {
    }
    
    @Override
    public void reset() {
    }

    @Override
    public AI clone() {
        return new RandomAI();
    }

    @Override
    public PlayerAction getAction(int player, GameState gs) {
        try {
            if (!gs.canExecuteAnyAction(player)) return new PlayerAction();
            PlayerActionGenerator pag = new PlayerActionGenerator(gs, player);
            return pag.getRandom();
        }catch(Exception e) {
            return new PlayerAction();
        }
    }

} 

SyntaxError: invalid syntax (<ipython-input-1-b5afe3394bb5>, line 1)

Zunächst schauen wir uns einfachere Agenten an. Komplexere Agenten besitzen neben allen Sachen die auch einfachere Agenten besitzen, noch deutlich komplexere Mechaniken, auf die im weiteren Verlauf noch eingegangen wird. Die einfachsten Agenten von MicroRTS sind ziemlich übersichtlich. Als Beispiel wird der Agenten **RandomAI** genutzt, der nur Random-Aktionen auswählen kann. Sofort zu erkennen sind einige überladene Funktionen, sowie eine Erweiterung des Agenten durch die **abstrakte Klasse AI**. AI ist die **Basisklasse für jeden MicroRTS-Agenten**, entweder direkt wie bei RandomAI, oder indirekt, über andere AI-Basisklassen, wie bei komplexeren Agenten. Die interessanteste Funktion bei allen Agenten ist die Funktion **public PlayerAction getAction (int player, GameState gs)**. Diese Funktion ist bei allen Agenten die Funktion die die verschiedenen Eigenschaften, Verhaltensweisen und Taktiken des Agenten, sowie den GameState miteinander verknüpft.

In [ ]:
public abstract class AI {
    public abstract void reset();

    public void reset(UnitTypeTable utt)  {reset();}

    public abstract PlayerAction getAction(int player, GameState gs) throws Exception;

    public abstract AI clone();

    public abstract List<ParameterSpecification> getParameters();

    public void preGameAnalysis(GameState gs, long milliseconds) throws Exception{}

    public void gameOver(int winner) throws Exception{}
}

In der AI-Basisklasse sind neben einigen Grundfunktionen, einige Funktionalitäten verpackt. Beispielsweise deklariert diese Abstrakte Klasse wichtige Funktionen wie getAction. Zwei Klassen sind bei erster Sicht sofort zu erkennen. Innerhalb des Konstruktors und der reset-Funktion wird die **Klasse UnitTypeTable** genutzt, und die Funktion **getAction** nutzt den **GameState**, sowie als Return-Typ die Klasse **PlayerAction**. Zusätzlich existiert eine Klasse Namens **ParameterSpecification**. Zunächst das UnitTypeTable.

In [ ]:
public class UnitTypeTable  {
...
    List<UnitType> unitTypes = new ArrayList<>();
...
    public void setUnitTypeTable(int version, int crs) {
    ...
    // Create the unit types:
        // RESOURCE:
        UnitType resource = new UnitType();
        resource.name = "Resource";
        resource.isResource = true;
        resource.isStockpile = false;
        resource.canHarvest = false;
        resource.canMove = false;
        resource.canAttack = false;
        resource.sightRadius = 0;
        addUnitType(resource);     
        // BASE:
        ...
        // BARRACKS: 
        ...
        // WORKER: 
        ...
        // LIGHT: 
        ...
        // HEAVY: 
        ...
        // RANGED: 
        ...
    }
    ...
}

Das **UnitTypeTable** speichert die Einheitentypen, die es im Spiel geben kann. Sie bestimmt auch die Eigenschaften der einzelnen Einheitentypen. Das UnitTypeTable bestimmt also das Gleichgewicht des Spiels (also wie stark Einheiten sind). Neben Funktionen, wie das Angebot ein UnitTypeTable per XML zu importieren oder zu exportieren und eine Menge weiterer Eigenschaften, sind die wichtigsten Funktionen von **void setUnitTypeTable (int version, int crs)**. Wie bereits an der Tabelle zu erkennen, existieren vorgebaute **UnitType** wie Worker, Base und Ressourcen, die abgebaut werden können. Als Entwickler kann man auch eigene Units einführen. Dabei ist jeder ein UnitType innerhalb der Klassenstruktur des Typen UnitType. Man kann also einfacher sagen das UnitTypeTable ist nichts Weiteres als **eine Array-Liste des Typen UnitType mit zusätzlichen Eigenschaften und Funktionalitäten**.

In [ ]:
public class UnitType {
    public int ID = 0; 
    public String name;
    public int cost = 1;
    public int hp = 1;
    public int minDamage = 1;
    public int maxDamage = 1;
    public int attackRange = 1;
    public int produceTime = 10, 
               moveTime = 10, 
               attackTime = 10, 
               harvestTime = 10, 
               returnTime = 10;
    ...
    public boolean canMove = true;  
    public boolean canAttack = true;  
    public ArrayList<UnitType> produces = new ArrayList<>();
    public ArrayList<UnitType> producedBy = new ArrayList<>();
    ...
    public void produces(UnitType ut) 
    {
        produces.add(ut);
        ut.producedBy.add(this);
    }
...
}

Die Klasse **UnitType** definiert mit seinen vielen Eigenschaften den abstrakten Typ einer Einheit. Neben den **Eigenschaften** wie Namen, hat diese Klasse **boolesche Eigenschaften**. Ob sich diese Einheit beispielsweise bewegen kann, sowie auch **Array Listen oder was diese Einheit für andere Einheiten produzieren** kann. Diese Klasse beinhaltet auch einige Funktionalitäten XML und JSON Dateienimportieren und exportieren zu können und diese zu Updaten. Jeder **Unit** wird genau einer UnitType zugesprochen. 

In [ ]:
public class Unit implements Serializable {
    UnitType type;
    public static long next_ID = 0;
    long ID;
    int player;
    int x, y;
    int resources;
    int hitpoints = 0;
    ...
    public boolean isIdle(GameState s) {
        UnitAction ua = s.getUnitAction(this);
        return ua == null;
    }

    public boolean canExecuteAction(UnitAction ua, GameState gs) {
        List<UnitAction> l = getUnitActions(gs, ua.ETA(this));
        return l.contains(ua);
    }

    public List<UnitAction> getUnitActions(GameState s, int noneDuration);
    ...
}

Die Klasse **Unit** stellt die Instanz einer beliebigen Einheit im Spiel dar. Sie definiert wie oben genannt, welchen Typ diese Einheit hat, zu welchem Spieler sie gehört, welche Position diese auf dem Spielfeld einnimmt, sowie weitere Eigenschaften. Neben vielen Hilfsfunktionen, sowie der Möglichkeit per XML zu importieren und zu exportieren sind die folgenden zwei Funktionen die wichtigsten dieser Klasse:   

1. public List<UnitAction> getUnitActions(GameState s, int noneDuration)
2. public boolean canExecuteAction(UnitAction ua, GameState gs)

Um diese Funktionen nachvollziehen zu können muss zunächst definiert werden, **was eine Aktion im Kontext von MicroRTS** überhaupt bedeutet.

<br>
<br>

#### 2. Aktionen für MicroRTS – Ein Abstraktes Aktion Layer

In [ ]:
public abstract class AbstractAction {
    
    Unit unit;
    
    public AbstractAction(Unit a_unit) {
        unit = a_unit;
    }
    ...
    public abstract boolean completed(GameState pgs);
    
    public UnitAction execute(GameState pgs){
    	return execute(pgs,null);
    }
    ...
    public abstract UnitAction execute(GameState pgs, ResourceUsage ru);

Wie beim vorherigen Teil zu erkennen ist, hat zunächst jede Unit eine Menge von **UnitActions**, die am Ende eine **Teilmenge der PlayerActions des Agenten** sind. Wie wird eine Abstrakte Aktion in MicroRTS definiert? Als einzige Eigenschaft hat die **Klasse AbstractAction** eine Instanz einer Unit. Im Allgemeinen kann man zwei wichtige Funktionen erkennen:

1. public abstract UnitAction execute(GameState pgs, ResourceUsage ru)
2. public abstract boolean completed(GameState pgs)

Die Funktion execute(…) definiert, **wie eine bestimmte Aktion auf der Gameoberfläche ausgeführt** wird.  Dagegen definiert die Funktion completed(…) **ob eine Aktion beendet worden ist**. Im Folgenden Bereich kann dies anhand einer **expliziten Aktion Move** nachvollziehen.


In [ ]:
public class Move extends AbstractAction {

    int x,y;
    PathFinding pf;

    
    public Move(Unit u, int a_x, int a_y, PathFinding a_pf) {
        super(u);
        x = a_x;
        y = a_y;
        pf = a_pf;
    }
    
    public boolean completed(GameState gs) {
        return unit.getX() == x && unit.getY() == y;
    }
    ...
    public UnitAction execute(GameState gs, ResourceUsage ru) {
        PhysicalGameState pgs = gs.getPhysicalGameState();
        UnitAction move = pf.findPath(unit, x+y*pgs.getWidth(), gs, ru);
        if (move!=null && gs.isUnitActionAllowed(unit, move)) return move;
        return null;
    }
}

Zunächst stellt sich die Frage welche Eigenschaften, für eine **Move-Aktion** gebraucht werden. Da sich bei **Move die Koordinaten der Einheit ändert sind die X und Y Koordinaten** als Eigenschaften, die die Koordinaten nach einer Bewegungsaktion halten, zwingend nötig. Beispielsweise die Attack-Aktion benötigt ein Angriffsziel was immer eine Unit sein wird. Am Beispiel von Move können auch execute und completed besser verstanden werden. In completed von Move wird nur getestet, ob die Unit, die in der super-Klasse gehalten wird, dieselben X und Y Koordinaten besitzt, wie die Klasse Move. Bei execute hingegen wird zunächst der **GameState** abgefragt, genauer der **PhysicalGameState**. Dann wird mithilfe des **Pathfinding** Algorithmus (Standard A*) die beste Position auf der Karte ausgesucht. Falls eine Position auf der Karte gefunden wurde, und diese auch gültig ist, gibt die Funktion eine **UnitAction** zurück , sonst null. Wichtigste Funktionen für die Aktionsfindung werden in dieser Klasse definiert.

In [ ]:
public class UnitAction {
    //codierung der Aktionstypen
    public static final int TYPE_NONE = 0;
    public static final int TYPE_MOVE = 1;
    public static final int TYPE_HARVEST = 2;
    public static final int TYPE_RETURN = 3;
    public static final int TYPE_PRODUCE = 4;
    public static final int TYPE_ATTACK_LOCATION = 5;
    public static final int NUMBER_OF_ACTION_TYPES = 6;
    public static String actionName[] = {
        "wait", "move", "harvest", "return", "produce", "attack_location"
    };

    //codierung der Bewegung
    public static final int DIRECTION_NONE = -1;
    public static final int DIRECTION_UP = 0;
    public static final int DIRECTION_RIGHT = 1;
    public static final int DIRECTION_DOWN = 2;
    public static final int DIRECTION_LEFT = 3;
    public static final String DIRECTION_NAMES[] = {"up", "right", "down", "left"};
    public static final int DIRECTION_OFFSET_X[] = {0, 1, 0, -1};
    public static final int DIRECTION_OFFSET_Y[] = {-1, 0, 1, 0};
    int parameter = DIRECTION_NONE; //direction & duration

    // weitere wichtige Eigenschaften für das erstellen einer UnitAction
    int x = 0, y = 0;      // X und Y Koordinate einer Attack-Location
    UnitType unitType;     // UnitType der für eine Train-Aktion gebraucht wird
    ResourceUsage r_cache; // Menge der mit dieser Aktion verbundenen Ressourcen
    ...
    public UnitAction(int a_type, int a_x, int a_y) {
        type = a_type;
        x = a_x;
        y = a_y;
    }
    ...
    public ResourceUsage resourceUsage(Unit u, PhysicalGameState pgs) {
        ...
    }
    public int ETA(Unit u) {
        switch (type) {
            case TYPE_NONE:
                return parameter;
            case TYPE_MOVE:
                return u.getMoveTime();
            case TYPE_ATTACK_LOCATION:
                return u.getAttackTime();
            case TYPE_HARVEST:
                return u.getHarvestTime();
            case TYPE_RETURN:
                return u.getMoveTime();
            case TYPE_PRODUCE:
                return unitType.produceTime;
        }
        return 0;
    }

    public void execute(Unit u, GameState s) {
        PhysicalGameState pgs = s.getPhysicalGameState();
        switch (type) {
            case TYPE_NONE:
                break;

            case TYPE_MOVE:
                switch (parameter) {
                    case DIRECTION_UP:
                        u.setY(u.getY() - 1);
                        break;
                    case DIRECTION_RIGHT:
                        u.setX(u.getX() + 1);
                        break;
                    case DIRECTION_DOWN:
                        u.setY(u.getY() + 1);
                        break;
                    case DIRECTION_LEFT:
                        u.setX(u.getX() - 1);
                        break;
                }
                break;
            case TYPE_ATTACK_LOCATION:
            ...
            case TYPE_HARVEST:
            ...
            case TYPE_RETURN:
            ...
            case TYPE_PRODUCE:
            ...   
        }
        break;
    }
    ...
}

Wie bereits an der Tabelle im zweiten Kapitel zu erkennen, werden die verschiedenen UnitAction codiert (string und int). Beispielsweise ist die UnitAction Move 1, und der Versuch auf eine bestimmte Location anzugreifen 6. Desweitern wird auch die Richtung bei Bewegung der Einheiten also up, down, right und left codiert (String und int). Dabei entsprechen die Indizes den Konstanten, die in dieser Klasse verwendet werden. Der durch die Bewegungsrichtung verursachter Offset wird zuletzt ebenfalls definiert. **Die 5 Konstruktoren stehen dabei für die 5 verschiedenen Aktionen**. Beispielsweise UnitTyp und Koordinaten für eine Train-Aktion. Neben Funktionalität zum Importieren und Exportieren von XML und JSON Dateien definiert auch diese Klasse wieviel Ressourcen für diese UnitAction gebraucht werden. Die beiden Wichtigen Funktion in dieser Klasse sind **public int ETA (Unit u)** und **public void execute (Unit u, GameState s)**. Die Funktion ETA gibt dabei die geschätzte Zeit des Abschlusses dieser Aktion zurück. Die Funktion execute nimmt dabei eine wichtige Rolle ein. Es wird zunächst geschaut welche UnitAction vorliegt. Bei einer Bewegung beispielsweise befindet man sich im Case TYPE_MOVE. Je nachdem in welche Richtung die Einheit die Move-Aktion ausführen will, werden vier weitere Cases für die Bewegungsrichtung ausgewählt. Auch für den Spieler gibt es einen Aktionsraum, dieser ist in der Klasse **PlayerAction** gekapselt.

In [ ]:
public class PlayerAction {
    List<Pair<Unit,UnitAction>> actions = new LinkedList<>();
    ResourceUsage r = new ResourceUsage();// Stellt die von der PlayerAction verwendeten Ressourcen dar

    public PlayerAction merge(PlayerAction a) {
        PlayerAction merge = new PlayerAction();
        merge.actions.addAll(actions);
        merge.actions.addAll(a.actions);
        merge.r = r.mergeIntoNew(a.r);
        
        return merge;
    }
    ...
}

Die **Klasse PlayerAction** verwaltet eine Liste von **Paaren aus Units und UnitActions**. Des Weiteren hat diese Klasse noch einiges an Funktionalität. Der erste Teil verwaltet die Menge der Aktionen, ob Non-Aktionen in der Liste sind. Sie weiß auch über eine andere Funktion, wie viele andere Aktionen in der Liste sind. Falls eine Unit keine Aktion zugewiesen bekommen hat, wird fillWithNones() diese mit Non-Aktionen befüllen. Eine weitere wichtige Funktion für die späteren Aktionsmenge ist die Funktion merge, die zwei PlayerAction zu einer kombiniert (auch von den Ressourcen). Jetzt muss dieses **Konstrukt aus Einheiten und Aktion mit den Agenten zu einem Layer** zusammengeführt werden. Dafür wird als Grundlage die **Klasse AbstractionLayerAI** genutzt.

In [ ]:
public abstract class AbstractionLayerAI extends AIWithComputationBudget {
    protected HashMap<Unit, AbstractAction> actions = new LinkedHashMap<>();
    protected PathFinding pf;
    protected GameState lastGameState;
    ...
    public void move(Unit u, int x, int y) {
        actions.put(u, new Move(u, x, y, pf));
    }
    public void train(Unit u, UnitType unit_type) {
        actions.put(u, new Train(u, unit_type));
    }
    public void build(Unit u, UnitType unit_type, int x, int y) {
        actions.put(u, new Build(u, unit_type, x, y, pf));
    }
    public void harvest(Unit u, Unit target, Unit base) {
        actions.put(u, new Harvest(u, target, base, pf));
    }
    public void attack(Unit u, Unit target) {
        actions.put(u, new Attack(u, target, pf));
    }
    public void idle(Unit u) {
        actions.put(u, new Idle(u));
    }

    public PlayerAction translateActions(int player, GameState gs) {        
        PhysicalGameState pgs = gs.getPhysicalGameState();
        PlayerAction pa = new PlayerAction();
        List<Pair<Unit, UnitAction>> desires = new ArrayList<>();

        lastGameState = gs;
        
        // Execute abstract actions:
        List<Unit> toDelete = new ArrayList<>();
        ResourceUsage ru = new ResourceUsage();
        for (AbstractAction aa : actions.values()) {
            if (!pgs.getUnits().contains(aa.unit)) {
                // The unit is dead:
                toDelete.add(aa.unit);
            } else {
                if (aa.completed(gs)) {
                    // the action is complete:
                    toDelete.add(aa.unit);
                } else {
                    if (gs.getActionAssignment(aa.unit) == null) {
                        UnitAction ua = aa.execute(gs, ru);
                        if (ua != null) {
                            if (VERIFY_ACTION_CORRECTNESS) {
                                // verify that the action is actually feasible:
                                List<UnitAction> ual = aa.unit.getUnitActions(gs);
                                if (ual.contains(ua)) {
                                    desires.add(new Pair<>(aa.unit, ua));
                                }
                            } else {
                                desires.add(new Pair<>(aa.unit, ua));
                            }
                            ru.merge(ua.resourceUsage(aa.unit, pgs));
                        }

                    }
                }
            }
        }
        for (Unit u : toDelete) {
            actions.remove(u);
        }

        // compose desires:
        ResourceUsage r = gs.getResourceUsage();
        pa.setResourceUsage(r);
        for (Pair<Unit, UnitAction> desire : desires) {
            ResourceUsage r2 = desire.m_b.resourceUsage(desire.m_a, pgs);
            if (pa.consistentWith(r2, gs)) {
                pa.addUnitAction(desire.m_a, desire.m_b);
                pa.getResourceUsage().merge(r2);
            }
        }

        pa.fillWithNones(gs, player, 10);
        return pa;
    }
    ...
    public int findBuildingPosition(List<Integer> reserved, int desiredX, int desiredY, Player p, PhysicalGameState pgs) {
    ...
    }
}

Zunächst sind einige neue Klassen in den Eigenschaften zu erkennen. Wie man sieht, bietet dieser Layer die Möglichkeiten **PathFinding-Algorithmen** einzubinden. Wie diese genau implementiert werden, wird im späteren Verlauf kurz gezeigt. Zu dieser Klasse reicht die Information, dass jede **„Highlevel-Aktion“** wenigstens A* nutzt. In MicroRTS sind folgende Highlevel-Aktionen definiert:


1. move (x, y) 
2. train (type)
3. build (type, x, y)
4. harvest (target)
5. attack (target)
Diese Aktionen werden als einfache Funktionen definiert, die eine Hashmap füllen. Innerhalb dieser HashMap werden Units zusammen mit AbstractAction zusammengehalten. Die wichtigste Funktion in dieser Klasse ist die Funktion **public PlayerAction translateActions (int player, GameState gs)**, die man sich näher anschauen sollte. Einfach gesagt wird in dieser Funktion alle bereits ausgeführten Einheitenaktionen zu einer PlayerAction zusammengefasst. Innerhalb dieser Funktion wird auch execute genutzt. In dieser Klasse wird auch definiert, wie eine **Building-Position gesucht** wird. Dies ist der grobe Unit- & Aktionsaufbau von MicroRTS. Schauen wir uns anhand eines **expliziten Agenten** an, wie alle diese Klassen zusammenarbeiten.

<br>
<br>

#### 3. Taktiken – Komplexere Agenten und dem Beginn von Strategie und Verhalten

In [ ]:
public class HeavyRush extends AbstractionLayerAI {
    Random r = new Random();
    protected UnitTypeTable utt;
    UnitType workerType;
    UnitType baseType;
    UnitType barracksType;
    UnitType heavyType;
    ...
    public HeavyRush(UnitTypeTable a_utt, PathFinding a_pf) {
        super(a_pf);
        reset(a_utt);
    }
    ...
    public void reset(UnitTypeTable a_utt)  
    {
        utt = a_utt;
        workerType = utt.getUnitType("Worker");
        baseType = utt.getUnitType("Base");
        barracksType = utt.getUnitType("Barracks");
        heavyType = utt.getUnitType("Heavy");
    }
    ...
    public PlayerAction getAction(int player, GameState gs) {
        PhysicalGameState pgs = gs.getPhysicalGameState();
        Player p = gs.getPlayer(player);
        // behavior of bases:
        for (Unit u : pgs.getUnits()) {
            if (u.getType() == 
                    && u.getPlayer() == player
                    && gs.getActionAssignment(u) == null) {
                baseBehavior(u, p, pgs); // hier ist der Übergang zum expliziten Agenten
            }
        }

        // behavior of barracks:
        for (Unit u : pgs.getUnits()) {
            if (u.getType() == barracksType
                    && u.getPlayer() == player
                    && gs.getActionAssignment(u) == null) {
                barracksBehavior(u, p, pgs); // hier ist der Übergang zum expliziten Agenten
            }
        }

        // behavior of melee units:
        ...
        return translateActions(player, gs);
    }
    ...
        public void baseBehavior(Unit u, Player p, PhysicalGameState pgs) {
        int nworkers = 0;
        for (Unit u2 : pgs.getUnits()) {
            if (u2.getType() == workerType
                    && u2.getPlayer() == p.getID()) {
                nworkers++;
            }
        }
        if (nworkers < 1 && p.getResources() >= workerType.cost) {
            train(u, workerType);
        }
    }

    public void barracksBehavior(Unit u, Player p, PhysicalGameState pgs) {
        if (p.getResources() >= heavyType.cost) {
            train(u, heavyType);
        }
    }
    ...
}

An diesen **komplexeren Agenten HeavyRush** kann man neben einigen Eigenschaften und den bereits oben genannten wichtigen Funktion getAction, weitere Funktionen mit dem Namen ...Behavoir erkennen. Zunächst **implementiert jeder komplexere Agent eine Taktik**. Zum Beispiel HeavyRush möchte so schnell wie möglich eine Baracke bauen und schwere Einheiten ausbilden, die direkt den Gegner angreifen sollen. Worker müssen dabei schnell die richtigen Gebäude bauen und den Spieler dauerhaft mit Ressourcen versorgen. Wie man bereits an den Eigenschaften erkennen kann, muss man **für die jeweilige Taktik die passenden UnitTypen vordefinieren** (werden meist auf UnitTypeTable gelesen). Wie der Agent **innerhalb der Taktik mit den verschiedenen UnitTypen** umgeht, wird in der **Behavior-Funktion** definiert. HeavyRush hat folgende Behavoir-Funktionen:



1. void baseBehavior (Unit u, Player p, PhysicalGameState pgs), 
2. void barracksBehavior (Unit u, Player p, PhysicalGameState pgs), 
3. void meleeUnitBehavior (Unit u, Player p, GameState gs), 
4. void workersBehavior (List<Unit> workers, Player p, GameState gs)

Genutzt werden diese Funktionen innerhalb der **Funktion PlayerAction getAction(int player, GameState gs)**. An dieser Stelle wird pro Behavior-Funktion jeweils eine for each Schleife programmiert. Die Funktion wird bei jedem Spielzyklus mit dem aktuellen Spielstand aufgerufen und gibt zurück, welche Aktion der Agent in diesem Zyklus ausführen will. Mithilfe dem **IF werden dabei nur die Units herausgezogen**, die für das jeweilige Verhalten gebraucht werden. Im ersten If werden die **Units durch den UnitType baseType aussortiert**. Am Ende der jeweiligen Schleifen wird in die **passende Behavior-Funktion gesprungen**. Sobald **alle Schleifen durch sind** , wird die Funktion **translateActions (player, gs) gecallt, die die Aktionen ausführt** und zu einer PlayerAction zusammenfasst.Schauen wir uns im nächsten Abschnitt die Klasse **GameState** genauer an.

<br>
<br>

#### 4. Die Umgebung – Status und Eigenschaften von MicroRTS

In [ ]:
public class GameState {
...
protected int time = 0;
protected UnitTypeTable utt;
protected PhysicalGameState pgs;
protected HashMap<Unit,UnitActionAssignment> unitActions = new LinkedHashMap<>();
protected UnitTypeTable utt;
...
public boolean free(int x, int y){
    ...
}
public boolean[][] getAllFree() {
    ...
}
...
public boolean canExecuteAnyAction(int pID) {
        for(Unit u : pgs.getUnits()) {
            if (u.getPlayer() == pID) {
                if (unitActions.get(u) == null) return true;
            }
        }
        return false;
}

public boolean isUnitActionAllowed(Unit u, UnitAction ua) {
...
// Generate the reserved resources:
...
}
...
public boolean cycle() {
        time++;
        
        List<UnitActionAssignment> readyToExecute = new LinkedList<>();
        for(UnitActionAssignment uaa:unitActions.values()) {
            if (uaa.action.ETA(uaa.unit)+uaa.time<=time) readyToExecute.add(uaa);
        }
                
        // execute the actions:
        for(UnitActionAssignment uaa:readyToExecute) {
            unitActions.remove(uaa.unit);  
            uaa.action.execute(uaa.unit,this);
        }
        
        return gameover();
}
...
public boolean integrityCheck() {
    ...
}
...
}


Der **GameState** besteht dabei aus **mehreren Teilen**. Der erste kleine Teil ist die Eigenschaft **int time**, die für ein **Echtzeit**strategiespiel notwendig ist.  Bei MircoRTS zeigt dieser Wert an, **wie viele Zyklen seit Beginn des Spiels vergangen sind**. Ein weiterer Wert ist das **UnitTypeTable**, das die Eigenschaften von Einheiten dieses Spieles speichert. Zwei Methoden, die in den Aktionen öfters genutzt werden, sind free und getAllfree. Mit den Hilfe dieser Funktionen getAllFree wird geschaut, ob sich Einheiten an der angegebenen Position befinden und keine Einheit eine Aktion durchführt, die diese Position nutzt. Zusätzlich werden mit **boolean isUnitActionAllowed (Unit u, UnitAction ua) und boolean canExecuteAnyAction (int pID) die Aktionen kontrolliert**. Die erste Funktion prüft, ob die vorgesehene UnitAction Konflikte mit einer anderen Action hat. Sie geht davon aus, dass die UnitAction ua gültig ist (d.h. eine der Aktionen, die die Unit potenziell ausführen kann). Die zweite Funktion hingegen prüft, ob ein Spieler in diesem Zustand eine Aktion ausführen kann. Um die Aktionen mit den Einheiten für den GameState aufzubereiten, speichert **UnitActionAssignment** die Einheiten mit der Zeit, die eine zugewiesene Aktion braucht.  Zuletzt ist die Funktion **boolean cycle ()** zuständig einen Spielzyklus und alle zugewiesenen Aktionen auszuführen. Viele weitere Funktionen werden für komplexere Techniken, wie zum Beispiel getPlayerActionsSingleUnit (Unit unit), bei UCT genutzt.  Als letzten Teil werden Eigenschaften der physikalischen Welt in einer weiteren Klasse gekapselt.

In [ ]:
public class PhysicalGameState {
    public static final int TERRAIN_NONE = 0; //free tile
    public static final int TERRAIN_WALL = 1; // blocked til
    int width = 8;
    int height = 8;
    int terrain[];
    List<Player> players = new ArrayList<>();
    List<Unit> units = new LinkedList<>();
    ...

    public PhysicalGameState(int a_width, int a_height) {
        width = a_width;
        height = a_height;
        terrain = new int[width * height];
    }

    public void addUnit(Unit newUnit) throws IllegalArgumentException {
        for (Unit existingUnit : units) {
            if (newUnit.getX() == existingUnit.getX() && newUnit.getY() == existingUnit.getY()) {
                throw new IllegalArgumentException(
                        "PhysicalGameState.addUnit: added two units in position: (" + newUnit.getX() + ", " + newUnit.getY() + ")");
            }
        }
        units.add(newUnit);
    }

    public void removeUnit(Unit u) {
        units.remove(u);
    }
    ...
    //sind in der Klasse ausprogrammiert!
    public boolean equivalents(PhysicalGameState pgs);
    public Unit getUnitAt(int x, int y);
    public boolean[][] getAllFree();
    public int winner();
    boolean gameover();
    public Collection<Unit> getUnitsAround(int x, int y, int squareRange);
    public Collection<Unit> getUnitsAround(int x, int y, int width, int height);
    public Collection<Unit> getUnitsInRectangle(int x, int y, int width, int height);
    ...

}

**PhysicalGameState** hält **die physikalischen Grenzen der Map** und die **Units, die auf der Map sind**. Wie groß diese Map ist, wird in dieser Klasse im Konstruktor definiert. Zudem weiß die Klasse, welche Spieler spielen. Damit implementiert sie auch die Funktionalität von winner () und gameover (), da diese Zustände Abhängig sind, ob die Spieler jeweils mindestens noch ein Spieler eine Unit auf dem Feld besitzen. Auch die Funktionalität, ob ein Feld frei oder bereits von einer anderen Unit besetzt wird, wird in dieser Klasse gekapselt. Weiterhin bietet die Klasse: 

1. Unit getUnitAt (int x, int y)
2. Collection<Unit> getUnitsAround (int x, int y, int squareRange) 
3. Collection<Unit> getUnitsInRectangle (int x, int y, int width, int height)

Mithilfe dieser Funktionen können **auf der Karte alle Einheiten in einem bestimmten Rechteck , oder Dreieck, wo im Zentrum die x und y Koordinate liegt, erfasst werden**. Diese Funktionen **helfen den Agenten später, die direkte Umgebung seiner Einheiten besser zu kennen, und damit analysieren zu können**. Wie eine neue Unit hinzugefügt oder eine besiegte entfernt werden soll wird auch in dieser Klasse definiert. Weiterhin gibt es auch einige clone () und equals () Funktionen, die unter anderem definieren wie ein Agent kopiert wird und wie GameStates miteinander verglichen werden. Zusätzlich sind große Mengen von XML und JSON bearbeitende Funktionen in dieser Klasse vertreten.

<br>
Übergang zu 5 ???
<br>
<br>

#### 5. Den Weg finden – PathFinding – Algorithmen
#### 6. Jetzt alles zusammen – sehr komplexe Agenten zusammen mit PPO
#### 7. Noch mehr Agenten – sehr komplexe Agenten mit verschiedener Algorithmik


Quelle des Codes : https://github.com/santiontanon/microrts

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9ce75853-4372-4d76-b47c-34c3ad90a24b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>